In [2]:
from sklearn.cluster import KMeans
import pandas as pd  
import numpy as np  
from sklearn.model_selection import GridSearchCV
from pandas import Series,DataFrame
%pwd

'd:\\BDSE26\\ML'

In [3]:
Bike = pd.read_csv("D:/BDSE26/ML/Top_Sites.csv",sep = ",")
Bike.columns()

C:\Users\j8003\AppData\Local\Temp\ipykernel_1268\4127884745.py:1: DtypeWarning: Columns (25,31,32) have mixed types. Specify dtype option on import or set low_memory=False.
  Bike = pd.read_csv("D:/BDSE26/ML/Top_Sites.csv",sep = ",")


MemoryError: Unable to allocate 166. MiB for an array with shape (2, 10862438) and data type object

In [4]:
Bike['sno'].astype('str')
Bike['Hr'].astype('str')

0           12
1            5
2           12
3           10
4           23
            ..
10862433    17
10862434    11
10862435     9
10862436    18
10862437    16
Name: Hr, Length: 10862438, dtype: object

In [5]:
ds= DataFrame(Bike)
X_choice = ds.loc[:, ['sno','Hr', 'holiday', 'school_off','rain', 'tot']]


In [6]:
kmeansModel = KMeans(n_clusters=13, random_state=46)
clusters_pred = kmeansModel.fit_predict(X_choice)
Bike ['k_meeans'] = clusters_pred

In [7]:
# # K-MEAN 畫圖
# kmeans_list = [KMeans(n_clusters=k, random_state=46).fit(X_choice)
#                 for k in range(1, 20)]
# inertias = [model.inertia_ for model in kmeans_list]

# from sklearn.metrics import silhouette_score
# silhouette_scores = [silhouette_score(X_choice, model.labels_)
#                      for model in kmeans_list[1:]]



In [8]:
# import matplotlib.pyplot as plt 
# plt.figure(figsize=(8, 3))
# plt.plot(range(2, 20,1), silhouette_scores, "bo-")
# plt.xlabel("$k$", fontsize=1)
# plt.ylabel("Silhouette score", fontsize=14)

# plt.show()

In [9]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
X_choice_reindex = Bike.reindex(columns=['sno', 'Hr', 'holiday', 'school_off', 'rain','Temperature','RH','WS','Precp','MRT_Dist','School_Dist','k_meeans','rent_count'])
X_choice_reindex

,sno,Hr,holiday,school_off,rain,Temperature,RH,WS,Precp,MRT_Dist,School_Dist,k_meeans,rent_count
0,80,12,0,0,0,25.5,66.0,4.2,0.0,934.143832,118.041403,0,9
1,80,5,1,1,0,21.5,87.0,1.2,0.0,934.143832,118.041403,0,2
2,80,12,1,1,0,13.6,85.0,4.5,0.0,934.143832,118.041403,0,7
3,80,10,0,0,0,14.9,76.0,4.0,0.0,934.143832,118.041403,0,7
4,80,23,1,1,0,13.7,57.0,3.7,0.0,934.143832,118.041403,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
10862433,500112084,17,1,1,0,20.5,87.0,1.0,0.0,336.624961,396.446516,4,3
10862434,500112084,11,0,0,0,30.6,61.0,1.9,0.0,336.624961,396.446516,4,2
10862435,500112084,9,0,0,0,28.4,71.0,0.0,0.0,336.624961,396.446516,4,1
10862436,500112084,18,0,0,0,21.2,87.0,1.9,0.0,336.624961,396.446516,4,2


In [10]:
ds = X_choice_reindex.values
X = ds[:,0:12]
y = ds[:,12]
y= y.astype('int64')
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20)

In [11]:
# 以下開始調整參數 https://medium.com/@pahome.chen/xgboost%E5%85%A5%E9%96%80%E7%B6%93%E9%A9%97%E5%88%86%E4%BA%AB-e06931b835f5
# 調整跌代次數n_estimators(樹木的數量)

cv_params = {'n_estimators': [100,200,300,400]}
other_params = {'learning_rate': 0.1, 
                'n_estimators': 100, 
                'max_depth': 5, 
                'min_child_weight': 1, 
                'seed': 0,
                'subsample': 0.8, 
                'colsample_bytree': 0.8, 
                'gamma': 0.1, 
                'reg_alpha': 0, 
                'reg_lambda': 1, 
                'tree_method':'gpu_hist'}

model = xgb.XGBRegressor(**other_params)
optimized_GBM = GridSearchCV(estimator=model, param_grid=cv_params, scoring='r2', cv=5, verbose=1, n_jobs=4)
optimized_GBM.fit(X_train, y_train)
evalute_result = optimized_GBM.cv_results_
print('参数的最佳取值：{0}'.format(optimized_GBM.best_params_))
print('最佳模型得分:{0}'.format(optimized_GBM.best_score_))
print('每輪迭代運行結果:{0}'.format(evalute_result))


Fitting 5 folds for each of 4 candidates, totalling 20 fits


d:\anaconda\lib\site-packages\joblib\externals\loky\process_executor.py:702: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


In [ ]:
# 調試參數gamma
cv_params = {'gamma': [0.1, 0.2, 0.3, 0.4, 0.5]}
other_params = {'learning_rate': 0.1, 
                'n_estimators': 400, 
                'max_depth': 5, 
                'min_child_weight': 1, 
                'seed': 0,
                'subsample': 0.8, 
                'colsample_bytree': 0.8, 
                'gamma': 0.2, 
                'reg_alpha': 0, 
                'reg_lambda': 1,
                'tree_method':'gpu_hist'}

model = xgb.XGBRegressor(**other_params)
optimized_GBM = GridSearchCV(estimator=model, param_grid=cv_params, scoring='r2', cv=5, verbose=1, n_jobs=4)
optimized_GBM.fit(X_train, y_train)
evalute_result = optimized_GBM.cv_results_
print('参数的最佳取值：{0}'.format(optimized_GBM.best_params_))
print('最佳模型得分:{0}'.format(optimized_GBM.best_score_))
print('每輪迭代運行結果:{0}'.format(evalute_result))

Fitting 5 folds for each of 5 candidates, totalling 25 fits


c:\Users\Student\anaconda3\lib\site-packages\joblib\externals\loky\process_executor.py:702: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


参数的最佳取值：{'gamma': 0.1}
最佳模型得分:0.6631713924314229
每輪迭代運行結果:{'mean_fit_time': array([135.34595814, 135.57572222, 136.37080078, 136.35264506,
       111.2853435 ]), 'std_fit_time': array([ 0.45034063,  0.79484365,  0.71629798,  0.92978293, 36.08994833]), 'mean_score_time': array([7.11056886, 7.19454408, 7.83881907, 7.0503294 , 5.81503997]), 'std_score_time': array([0.04797861, 0.04872627, 0.53935364, 0.72570801, 0.69612001]), 'param_gamma': masked_array(data=[0.1, 0.2, 0.3, 0.4, 0.5],
             mask=[False, False, False, False, False],
       fill_value='?',
            dtype=object), 'params': [{'gamma': 0.1}, {'gamma': 0.2}, {'gamma': 0.3}, {'gamma': 0.4}, {'gamma': 0.5}], 'split0_test_score': array([0.66432258, 0.66432258, 0.66432258, 0.66176165, 0.6608427 ]), 'split1_test_score': array([0.66390511, 0.66390511, 0.66311943, 0.66390511, 0.66390511]), 'split2_test_score': array([0.66307078, 0.66307078, 0.66307078, 0.66307078, 0.66307078]), 'split3_test_score': array([0.66366248, 0.6636

In [ ]:
# 調試參數max_depth(樹的深度)，min_child_weight(決定最小葉子節點樣本權重和)
cv_params = {'max_depth': [3, 4, 5, 6, 7], 'min_child_weight': [1, 2, 3, 4]}
other_params = {
    'learning_rate': 0.1,
    'n_estimators': 400,
    'max_depth': 7,
    'min_child_weight': 3,
    'seed': 0,
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    'gamma': 0.1,
    'reg_alpha': 0,
    'reg_lambda': 1,
    'tree_method': 'gpu_hist'
}

model = xgb.XGBRegressor(**other_params)
optimized_GBM = GridSearchCV(estimator=model, param_grid=cv_params, scoring='r2', cv=5, verbose=1, n_jobs=4)
optimized_GBM.fit(X_train, y_train)
evalute_result = optimized_GBM.cv_results_
print('参数的最佳取值：{0}'.format(optimized_GBM.best_params_))
print('最佳模型得分:{0}'.format(optimized_GBM.best_score_))
print('每輪迭代運行結果:{0}'.format(evalute_result))

Fitting 5 folds for each of 20 candidates, totalling 100 fits
参数的最佳取值：{'max_depth': 7, 'min_child_weight': 3}
最佳模型得分:0.7171657572069856
每輪迭代運行結果:{'mean_fit_time': array([104.10735717, 102.01189308,  99.77676244,  99.62525439,
       112.74824266, 113.14923096, 113.08975024, 112.42749896,
       129.88417349, 126.26037211, 128.39246655, 124.40439181,
       150.44764562, 146.65513859, 146.40507779, 143.2942946 ,
       173.53783617, 169.76585288, 171.01835179, 154.6605248 ]), 'std_fit_time': array([ 1.23736084,  4.24424199,  6.63124899,  5.11498181,  3.37238464,
        2.59075824,  1.79381753,  1.55208037,  3.68521165,  3.39386648,
        2.66884718,  3.36557858,  5.04854489,  1.97468768,  2.67825488,
        2.96842558,  4.86780619,  2.41762205,  2.30588327, 16.68511541]), 'mean_score_time': array([3.77828717, 3.8434248 , 3.7658217 , 3.55099754, 4.68227038,
       4.53167105, 4.71458125, 4.63100467, 5.81483874, 6.13246751,
       5.84302635, 5.95013947, 7.20232258, 7.31891494, 7.2584

In [ ]:
# 調subsample(值在 0 與 1 之間, 逐次匯入小部分數據以控制 overfitting 狀況)、
# olsample_bytree(一旦遇到 overfitting 就降低這些參數)
cv_params = {'subsample': [0.6, 0.7, 0.8, 0.9], 'colsample_bytree': [0.6, 0.7, 0.8, 0.9]}
other_params = {
    'learning_rate': 0.1,
    'n_estimators': 400,
    'max_depth': 7,
    'min_child_weight': 3,
    'seed': 0,
    'subsample': 0.7,
    'colsample_bytree': 0.9,
    'gamma': 0.1,
    'reg_alpha': 0,
    'reg_lambda': 1,
    'tree_method': 'gpu_hist'
}

model = xgb.XGBRegressor(**other_params)
optimized_GBM = GridSearchCV(estimator=model, param_grid=cv_params, scoring='r2', cv=5, verbose=1, n_jobs=4)
optimized_GBM.fit(X_train, y_train)
evalute_result = optimized_GBM.cv_results_
print('参数的最佳取值：{0}'.format(optimized_GBM.best_params_))
print('最佳模型得分:{0}'.format(optimized_GBM.best_score_))
print('每輪迭代運行結果:{0}'.format(evalute_result))

Fitting 5 folds for each of 16 candidates, totalling 80 fits
参数的最佳取值：{'colsample_bytree': 0.9, 'subsample': 0.7}
最佳模型得分:0.7195151158506518
每輪迭代運行結果:{'mean_fit_time': array([182.11775713, 180.85700388, 176.61266794, 172.97434516,
       165.71569672, 171.61108761, 171.75814767, 172.84220071,
       171.89076624, 165.48679671, 171.44631958, 173.91609087,
       174.02403765, 170.30155106, 164.01101398, 144.00916367]), 'std_fit_time': array([ 0.75454126,  1.1734086 ,  3.62281784, 10.16577101, 14.92873093,
       12.97435053, 11.30118119,  5.3505128 , 11.51027273, 16.69195549,
       14.27068416,  6.91780369,  3.43125778, 12.09552815, 18.02977768,
       50.26185135]), 'mean_score_time': array([11.89476299, 12.46842618, 10.34890032, 10.7159236 ,  9.8020956 ,
       10.45074716,  9.80006351,  9.99993954, 10.22972226, 10.4446681 ,
       10.2692152 , 11.12342672, 10.2658299 , 10.07843204,  9.83707037,
       10.02587032]), 'std_score_time': array([0.59660857, 1.55803639, 1.08929113, 1.232251

In [ ]:
# 正則化參數調優(損失函數的懲罰項)
# reg_alpha L1正則化權重項，增加此值將使模型更加保守
# reg_lambda L2正則化權重項，增加此值將使模型更加保守
cv_params = {'reg_alpha': [0.05, 0.1, 1, 2], 'reg_lambda': [0.05, 0.1, 1, 2]}
other_params = {
    'learning_rate': 0.1,
    'n_estimators': 400,
    'max_depth': 7,
    'min_child_weight': 3,
    'seed': 0,
    'subsample': 0.7,
    'colsample_bytree': 0.9,
    'gamma': 0.1,
    'reg_alpha': 0,
    'reg_lambda': 1,
    'tree_method': 'gpu_hist'
}

model = xgb.XGBRegressor(**other_params)
optimized_GBM = GridSearchCV(estimator=model, param_grid=cv_params, scoring='r2', cv=5, verbose=1, n_jobs=4)
optimized_GBM.fit(X_train, y_train)
evalute_result = optimized_GBM.cv_results_
print('参数的最佳取值：{0}'.format(optimized_GBM.best_params_))
print('最佳模型得分:{0}'.format(optimized_GBM.best_score_))
print('每輪迭代運行結果:{0}'.format(evalute_result))

Fitting 5 folds for each of 16 candidates, totalling 80 fits


In [ ]:
#learning_rate(指定學習率) 調優
#著迭代次數增加，會越來越靠近最小值。
# 學習率衰減的基本思想是學習率隨著訓練的進行逐漸衰減，
# 即在開始的時候使用較大的學習率，加快靠近最小值的速度，
# 在後來些時候用較小的學習率，提高穩定性，避免因學習率太大跳過最小值，
# 保證能夠收斂到最小值。
cv_params = {'learning_rate': [0.01, 0.05, 0.07, 0.1, 0.2]}
other_params = {'learning_rate': 0.2, 
                'n_estimators': 650, 
                'max_depth': 3, 
                'min_child_weight': 6, 
                'seed': 0,
                'subsample': 0.8, 
                'colsample_bytree': 0.9, 
                'gamma': 0.1, 
                'reg_alpha': 0.1, 
                'reg_lambda': 2,
                'tree_method':'gpu_hist'}

model = xgb.XGBRegressor(**other_params)
optimized_GBM = GridSearchCV(estimator=model, param_grid=cv_params, scoring='r2', cv=5, verbose=1, n_jobs=-1)
optimized_GBM.fit(X_train, y_train)
evalute_result = optimized_GBM.cv_results_
print('参数的最佳取值：{0}'.format(optimized_GBM.best_params_))
print('最佳模型得分:{0}'.format(optimized_GBM.best_score_))
print('每輪迭代運行結果:{0}'.format(evalute_result))

Fitting 5 folds for each of 5 candidates, totalling 25 fits
参数的最佳取值：{'learning_rate': 0.2}
最佳模型得分:0.6591492879358707
每輪迭代運行結果:{'mean_fit_time': array([5.80031829, 7.3505404 , 7.19617872, 7.11349072, 5.79429717]), 'std_fit_time': array([1.01917267, 0.94871971, 0.7084457 , 0.52479445, 1.65564392]), 'mean_score_time': array([0.12801895, 0.13344336, 0.17011361, 0.11469321, 0.09631562]), 'std_score_time': array([0.03546302, 0.01781172, 0.05000505, 0.03541877, 0.01168358]), 'param_learning_rate': masked_array(data=[0.01, 0.05, 0.07, 0.1, 0.2],
             mask=[False, False, False, False, False],
       fill_value='?',
            dtype=object), 'params': [{'learning_rate': 0.01}, {'learning_rate': 0.05}, {'learning_rate': 0.07}, {'learning_rate': 0.1}, {'learning_rate': 0.2}], 'split0_test_score': array([0.59036662, 0.63972806, 0.6457703 , 0.65049926, 0.65686079]), 'split1_test_score': array([0.58736217, 0.64147832, 0.64630125, 0.65291528, 0.65925044]), 'split2_test_score': array([0.588758

In [ ]:
#顯示重要特徵
from xgboost import plot_importance
from matplotlib import pyplot as plt
plot_importance(model)
plt.show()